In [33]:
import numpy as np
import cv2 as cv
import glob

In [34]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

In [35]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


In [36]:
images = glob.glob('calibration/*.jpg')

In [37]:
len(images)

6

In [38]:
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
 
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
 
        # Draw and display the corners
        #cv.drawChessboardCorners(img, (7,6), corners2, ret)
        #cv.imshow('img', img)
        #cv.waitKey(10000)
        #cv.destroyAllWindows()

In [47]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(np.asarray(rvecs).shape)
print(np.asarray(tvecs).shape)
np.savetxt('cameraMatrix.array',mtx)
np.savetxt('distanceCoefficients.array',dist)
np.savetxt('rotation.array',np.asarray(rvecs).reshape(1, -1))
np.savetxt('translation.array',np.asarray(tvecs).reshape(1,-1))

(1, 3, 1)
(1, 3, 1)


In [49]:
mtx=np.loadtxt('cameraMatrix.array')
dist=np.loadtxt('distanceCoefficients.array')
rvecs=np.loadtxt('rotation.array').reshape(1,3,1)
tvecs=np.loadtxt('translation.array').reshape(1,3,1)
img = cv.imread('calibration/undistorted/ch1-1765738034.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
cv.imshow('calibresult', dst)
cv.waitKey(30000)
cv.destroyAllWindows()